### View the current logical CPU count of the server

In [1]:
import os

NUM_CPU = len(os.sched_getaffinity(0)) #os.cpu_count() 

print(f'CPU total: {NUM_CPU}')

CPU total: 128


### Limit the number of threads that can be called by a single process

In [2]:
NUM_THREADS = 4

os.environ["MKL_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["NUMEXPR_NUM_THREADS"] = str(NUM_THREADS)
os.environ["OMP_NUM_THREADS"]     = str(NUM_THREADS)

NUM_PROCESS = NUM_CPU // NUM_THREADS
print(f'Maximum number of parallel processes: {NUM_PROCESS}')

Maximum number of parallel processes: 32


### Import numpy, multiprocessing and other packages

In [ ]:
pip install statsmodels

In [4]:
import numpy as np
import math
from numpy.random import default_rng
from time import time
import multiprocessing as mp
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import linear_model  
import random
import math

### Global invariant parameters

In [5]:
K_tru = 5; q = 8;
sigma = 1; theta = [3,1.5,0,0,2,0,0,0]; gamma = list(range(-4,3*K_tru-4,3));
pi = [0.15,0.2,0.3,0.25,0.1]
mean = np.zeros(q)
rho = 0.5
i, j = np.mgrid[:q, :q]
cov = rho**abs(i-j)

### Some functions needed for global calculations

In [6]:
def dup_rows(a, indx, num_dups=1):
    return np.insert(a,[indx+1]*num_dups,a[indx],axis=0)

def dup_cols(a, indx, num_dups=1):
    return np.insert(a,[indx+1]*num_dups,a[:,[indx]],axis=1)
def function_exp(x):
    return np.exp(x)
function_vexp = np.vectorize(function_exp)
def function_bin(p,x):
    return (p**x)*(1-p)**(1-x)
function_vbin = np.vectorize(function_bin)
def function_binlog(p,x):
    return (x*np.log(p)+(1-x)*np.log(1-p))
function_vbinlog = np.vectorize(function_binlog)

### EM

In [7]:
#Section 2.2-The initial estimator-EM
def em_single_initial(n,K,priors,X,Y):
    '''
    EM
    Arguments:
    priors:[pi_t,gamma_t,theta_t,sigma_t]
    Y:[n X 1 list]
    X:[n X q matrix]
    
    Returns:
    new_priors:[new_pi,new_gamma,new_theta,new_sigma]
    pi_t = priors[1:K]; gamma_t = priors[K:2*K]
    theta_t = priors[2*K:2*K+q]; sigma_t = priors[-1]
    '''
    pi_t = priors[0:K]; gamma_t = priors[K:2*K]
    theta_t = priors[2*K:2*K+q]; sigma_t = priors[-1]
    #E step -w_ik
    c = Y-np.dot(X,theta_t)
    g = np.array(gamma_t)
    g_pi = np.array(pi_t)
    a1 = (np.ones((K,n))*c.T).T
    a2 = np.ones((n,K))*g
    e_pri = -(a1-a2)**2/(2*sigma_t**2)
    e_pri = function_vexp(e_pri)
    a_pi = np.ones((n,K))*g_pi
    w_t1 = (np.divide((e_pri*a_pi).T,(np.sum(e_pri*a_pi,1)).T)).T

    #M step 
    new_pi = np.average(w_t1, axis=0) 
    weight = np.divide(w_t1,sum(w_t1,0))
    new_gamma = np.dot(weight.T, Y-np.dot(X, theta_t))
    V = np.dot(w_t1,gamma_t)
    trt_inv = np.dot(np.linalg.inv(np.dot(X.T,X)+np.eye(q)*0.001),X.T)
    new_theta = np.dot(trt_inv,Y-V)
    sum_sigma = 0
    
    w_t1_gamma_t = np.dot(w_t1,gamma_t)
    sum_sigma = {np.dot(((Y-np.dot(X,theta_t))**2).T,[1]*n)-
              2*np.dot(np.multiply(Y-np.dot(X,theta_t),w_t1_gamma_t).T,[1]*n)+
                np.dot(np.dot(w_t1,np.array(gamma_t)**2),[1]*n)}
    new_sigma = (list(sum_sigma)[0]/n)**0.5
    return list(new_pi)+list(new_gamma)+list(new_theta)+[new_sigma]

In [8]:
def initial_em(n,K,prior,X,Y,tol = 1e-3,iterations=10000):
    '''
    EM
    param Y,X :Data
    param prior：Initial
    param tol：End of Iteration Threshold
    param iterations：Maximum number of iterations
    return：Locally optimal model parameters
    '''
    iteration = 0;
    while iteration < iterations:
        new_prior = em_single_initial(n,K,prior,X,Y)
        delta_change = abs(np.array(prior)-np.array(new_prior))
        if sum(delta_change**2)**0.5<tol:
            break
        else:
            prior = new_prior
            iteration +=1
    return [new_prior,iteration]

In [9]:
def em_single_for_p(n,p,K,initial,prior_p,X,Y,Z):
    '''
    EM
    Arguments:
    initial: [pi_h,gamma_h,theta_h,sigma_h]
    priors:pj [k X 1 list]
    Y:[n X 1 list]
    X:[n X q matrix]
    Z:[n X p matrix]
    j: for p
    
    Returns:
    new_priors:new_pj
    '''
    pi_h = initial[0:K]; gamma_h = initial[K:2*K]
    theta_h = initial[2*K:2*K+q]; sigma_h = initial[-1]
    p_t = prior_p
    #E step -pi_ik

    c = Y-np.dot(X,theta_h)
    g = np.array(gamma_h)
    g_pi = np.array(pi_h)
    a1 = dup_cols(np.column_stack((c,c)), indx=0, num_dups=K-2)
    a2 = dup_rows(np.row_stack((g,g)), indx=0, num_dups=n-2)
    e_pri = -(a1-a2)**2/(2*sigma_h**2)
    e_pri = function_vexp(e_pri)
    eb_pri = np.zeros((K,n,p))
    for k in range(K):
        p_k = p_t[k,:]
        p_k_ma = np.ones((n,p))*p_k
        prod_k = p_k_ma*Z+(1-p_k_ma)*(1-Z)
        eb_pri[k,:,:] = pi_h[k]*((np.ones((p,n))*e_pri[:,k]).T)*prod_k
    pi_ik_j = np.zeros((K,n,p))
    new_p = np.zeros((K,p))
    dd = np.sum(eb_pri,0)
    for k in range(K):
        pi_ik_j[k,:,:] = np.divide(eb_pri[k,:,:],dd)
        weight_k = np.divide(pi_ik_j[k,:,:],np.sum(pi_ik_j[k,:,:],0))
        #M step
        new_p[k,:] = np.diag(np.dot(weight_k.T,Z))

    return new_p

In [10]:
def em_for_p(n,p,K,initial,prior_p,X,Y,Z,tol = 1e-6,iterations=100):
    '''
    EM
    param Y,X,Z :Data
    param prior：Initial value
    param initial: Other parameters
    param tol：End of Iteration Threshold
    param iterations：Maximum number of iterations
    return：Locally optimal model parameters
    '''
    iteration = 0;
    #prior_p = (prior_p+0.001)/1.001
    while iteration < iterations:
        new_prior_p = em_single_for_p(n,p,K,initial,prior_p,X,Y,Z)
        p_change = (prior_p-new_prior_p)**2
        err_norm = np.mean((np.sum(p_change,1))**0.5)
        if err_norm<tol:
            break
        else:
            prior_p = new_prior_p
            iteration +=1
            #print(iteration, err_norm)
    return [new_prior_p,iteration]

### 1. Simulation Data Generator--X,Y,Z

In [11]:
def data_generator(n,p,K,rho_kj,seed):

    rng = default_rng(seed) 
    #X
    X = rng.multivariate_normal(mean, cov, (n,), 'raise')   # nxq
    mk_class = rng.multinomial(n, pvals=pi)
    #mK_gamma
    mK_gamma = []
    mK = []
    for k in range(K):
        idt = np.ones(int(mk_class[k]))
        mK.extend(idt*(k))
        mK_gamma.extend(idt*gamma[k])
    mK = [int(k) for k in mK]
    #Y
    epsilon = list(rng.normal(size=n))
    Y = mK_gamma + np.dot(X, theta) + epsilon
    #Z
    Z = np.zeros((n,p))
    for k in range(K):
        ki_ind = [i for i,x in enumerate(mK) if x==k]
        for j in range(p):
            Z[ki_ind,j] = rng.binomial(1,rho_kj[k,j],len(ki_ind))
            
    return [X, Y, Z]

### 2. Function to compute the initial value of the initial-estimator

In [12]:
def initial_pri_est(n,K,X,Y):
    
    X_s = sm.add_constant(X)
    model = sm.OLS(Y, X_s)
    model_fit = model.fit()
    model_res = model_fit.resid
    c = model_res
    clf = KMeans(n_clusters=K)
    model_res = model_res.reshape(-1,1)
    ff = clf.fit(model_res)
    classgamma = ff.cluster_centers_
    classgamma = [x[0] for x in classgamma]
    classgamma.sort()
    gamma_pri = [x+model_fit.params[0] for x in classgamma]
    theta_pri = model_fit.params[1:]
    #pi_pri
    sample_label = pd.DataFrame({'value':c, 'label_f':ff.fit_predict(model_res),
                         'center':np.zeros(n), 'label':ff.fit_predict(model_res)})
    sample_label.sort_values(by = 'label_f')
    c = []
    for k in range(K):
        idx_k = sample_label[sample_label['label_f']==k].index.tolist()
        sample_label.loc[idx_k,'center'] = np.mean(sample_label.loc[idx_k,'value'])
        c.append(np.mean(sample_label.loc[idx_k,'value']))
   
    c.sort()
    for k in range(K):
        idx_k = sample_label[sample_label['center']==c[k]].index.tolist()
        sample_label.loc[idx_k,'label'] = k
        sample_label.sort_values(by = 'label')
    counts = sample_label.label.value_counts()/n
    pi_pri = []
    for k in range(K):
        pi_pri.append(counts.loc[k])
    cs = [1]*K
    for k in range(K):
        idx_k = sample_label[sample_label['label']==k].index.tolist()
        cs[k] = np.std(sample_label.loc[idx_k,'value'])
    sigma_pri = np.mean(cs)
    param_pri = pi_pri+gamma_pri+list(theta_pri)+[sigma_pri]
        
    return param_pri

### 3. p_kj prior

In [13]:
def p_pri_est(n,p,K,initial_est, X, Y, Z):
    #initial estimator
    pi_ini_est = initial_est[0:K]; gamma_ini_est = initial_est[K:2*K]
    theta_ini_est = initial_est[2*K:2*K+q]; sigma_ini_est = initial_est[-1]
    
    #Compute p_kj
    c = Y-np.dot(X,theta_ini_est)
    g = np.array(gamma_ini_est)
    g_pi = np.array(pi_ini_est)
    a1 = (np.ones((K,n))*c.T).T
    a2 = np.ones((n,K))*g
    e_pri = -(a1-a2)**2/(2*sigma_ini_est**2)
    e_pri = function_vexp(e_pri)
    a_pi = np.ones((n,K))*g_pi
    pi_ik_pri = (np.divide((e_pri*a_pi).T,(np.sum(e_pri*a_pi,1)).T)).T
    weight = np.divide(pi_ik_pri,sum(pi_ik_pri,0))
    p_pri = np.dot(weight.T, Z)
        
    return p_pri

### 4. Compute BIC

In [1]:
def BIC_for_K(n,p,K,initial_est, p_est, X,Y,Z):
    #posterior  
    df = 2*K+q+p*K
    pi_h = initial_est[0:K]; gamma_h = initial_est[K:2*K]
    theta_h= initial_est[2*K:2*K+q]; sigma_h = initial_est[-1]
    
    c = Y-np.dot(X,theta_h)
    g = np.array(gamma_h)
    g_pi = np.array(pi_h)
    a1 = (np.ones((K,n))*c.T).T
    a2 = np.ones((n,K))*g
    e_pri = -(a1-a2)**2/(2*sigma_h**2)
    e_pri = function_vexp(e_pri)/((2*math.pi*sigma_h**2)**0.5)
    
    eb_pri = np.zeros((K,n,p))
    for k in range(K):
        p_k = p_est[k,:]
        p_k_ma = np.ones((n,p))*p_k
        prod_k = p_k_ma*Z+(1-p_k_ma)*(1-Z)
        eb_pri[k,:,:] = pi_h[k]*((np.ones((p,n))*e_pri[:,k]).T)*prod_k 
    BIC_k = df*np.log(n)-2*np.sum(np.log(np.sum(eb_pri,axis=0)))
    
    return BIC_k

### 5. Define a mapping: random number seed$\mapsto$p estimator

In [198]:
def map_fun(b):
 
    X,Y,Z = data_generator(n,p,K_tru,rho_kj, seed = b) # generate data
    
    BIC = pd.DataFrame({'K':list(range(1,11)),'BIC': np.zeros(10)})
    for K in range(1,11):
        initial_pri = initial_pri_est(n, K, X, Y)                      #Compute initial pri
        initial_est, initial_iter = initial_em(n,K, initial_pri, X, Y) #Compute initial est
        p_pri = p_pri_est(n,p,K,initial_est, X, Y, Z)                  #Compute pkj pri
        p_est, p_iter = em_for_p(n,p,K,initial_est,p_pri,X, Y, Z)      #Compute pkj est
        #print(K,p_iter)
        BIC.iloc[K-1,1] = BIC_for_K(n,p,K,initial_est, p_est,X,Y,Z)
    
    re = np.array(BIC[BIC['BIC']==np.min(BIC['BIC'])])
    #print(BIC)
    print(re)
    return  re #返回BIC最小的K和值

### 6. Setting for simulation

In [362]:
n = 2000; p = 1000;  
B = 500 

In [363]:
#1 Generate p_kj
rd = np.random.RandomState(888) 
rho_kj = rd.uniform(0.01,0.3,(K_tru,p))
def function_2(x):
    return round(x,2)
function_vector = np.vectorize(function_2)
rho_kj = function_vector(rho_kj)
diag_pk = rd.uniform(0.8,0.95,int(p/K_tru))
diag_pk = function_vector(diag_pk)
for k in range(K_tru):
    np.random.shuffle(diag_pk)
    rho_kj[k,k*int(p/K_tru):(k+1)*int(p/K_tru)] = diag_pk

### 7. Calling multiple processes for simulation experiments

In [ ]:
tic1 = time()

with mp.Pool(NUM_PROCESS) as pool: 
    Results1 = pool.map(map_fun, range(B))

toc1 = time()
print(toc1 - tic1) # Total Time

### 8. Obtain Results

In [366]:
op_K_nn_n2000_p1000_B500 = np.zeros(B)
op_BIC_nn_n2000_p1000_B500  = np.zeros(B)
for b in range(B):
    op_K_nn_n2000_p1000_B500[b] = Results1[b][0][0]
    op_BIC_nn_n2000_p1000_B500[b] = Results1[b][0][1]

In [367]:
pd.DataFrame({'K':op_K_nn_n2000_p1000_B500,'BIC':op_BIC_nn_n2000_p1000_B500}).to_csv("nn_op_select/op_n2000_p1000_B500.csv",index=False)